In [1]:
import numpy as np
import pandas as pd
import time

### parameters

#### data urls

In [2]:
stockholm_2019_url = '/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/ld_stkhlm_data_20200510_exp.csv'
# stockholm_2018_url = '/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/ld_stkhlm_data_2018_ONLY__20200514_exp.csv'

curr_in_data_url = stockholm_2019_url

In [3]:
# uptime url 
uptime_path = "/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/uptime_url.csv"

In [4]:
#### variable(s) we're working on 
working_on_col_name = 'p2'

#### input / output tables 

In [5]:
df_sensorData = -1 # this is where we load the input data

In [6]:
# this where we insert the same-length data sets for the sensors 
# - useful for holding sensor data 
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean = -1
# median version
df_sensorids_columns_w_sensor_readings_2pt5_mins__median = -1

# as above but in 60 mins - using averages 
df_sensorids_columns_w_sensor_readings_60_mins__mean = -1 

# as above but in 60 mins - using median values 
df_sensorids_columns_w_sensor_readings_60_mins__median = -1 

In [7]:
# holds uptime statistics per sensor and time
# - same structure and order as above table, 
df_sensorids_columns_w_sensor_uptime_info_2pt5_mins = -1 

# as above but in hour frequency 
df_sensorids_columns_w_sensor_uptime_info_60_mins = -1 

In [8]:
# unique sensor ids 
unique_sensor_ids = -1 

# selected sensors?
selected_sensors = -1 

In [9]:
### secondary vars

# lat_lon_of_sensors
sensor_lat_lon = -1 

# selected sensors?
selected_sensors = -1

#### timing 

In [10]:
## the time period we're working with here 
# start_timestamp = '2018-01-01 00:00:00'
# end_timestamp = '2019-01-01 00:00:00'

start_timestamp = '2019-01-01 00:00:00'
end_timestamp = '2020-01-01 00:00:00'


# possible sample freuencies 
sensor_uptime_sampling_freq = "150S"
output_table_final_freq = "3600S"

# how many sensor periods are there in an hour?
sensor_periods_in_hour = 24


In [11]:
# relevant time ranges 
time_date_period_as_date_range__2pt5_mins = -1 
time_date_period_as_date_range__60_mins = -1 

In [12]:
# blank data rows, with the beginning and end dates 

# - good for equalling out the gathered sensor data time series 
in_data__START_TIME__blank_row = pd.DataFrame( data={"p1": np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [start_timestamp] ) )

in_data__END_TIME__blank_row = pd.DataFrame( data={"p1": np.NaN, "p2" : np.NaN }, index=pd.DatetimeIndex( [end_timestamp] ) )

### run! 

#### setup in/out data frames to the right length

In [13]:
# fetch relevant time-date ranges - 2.5 mins 
time_date_period_as_date_range__2pt5_mins = pd.date_range( start=start_timestamp, end=end_timestamp, freq=sensor_uptime_sampling_freq )

In [14]:
time_date_period_as_date_range__2pt5_mins.shape

(210241,)

In [15]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean

-1

In [16]:
# date range for 60 min sampling rates
time_date_period_as_date_range__60_mins = pd.date_range( start=start_timestamp, end=end_timestamp, freq=output_table_final_freq )
time_date_period_as_date_range__60_mins

DatetimeIndex(['2019-01-01 00:00:00', '2019-01-01 01:00:00',
               '2019-01-01 02:00:00', '2019-01-01 03:00:00',
               '2019-01-01 04:00:00', '2019-01-01 05:00:00',
               '2019-01-01 06:00:00', '2019-01-01 07:00:00',
               '2019-01-01 08:00:00', '2019-01-01 09:00:00',
               ...
               '2019-12-31 15:00:00', '2019-12-31 16:00:00',
               '2019-12-31 17:00:00', '2019-12-31 18:00:00',
               '2019-12-31 19:00:00', '2019-12-31 20:00:00',
               '2019-12-31 21:00:00', '2019-12-31 22:00:00',
               '2019-12-31 23:00:00', '2020-01-01 00:00:00'],
              dtype='datetime64[ns]', length=8761, freq='3600S')

#### now to make those tables… to hold the various sensor and uptime data 

In [17]:
# sensor readings at 2.5 mins 
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )
df_sensorids_columns_w_sensor_readings_2pt5_mins__median = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )

# sensor readings at 60 mins 
df_sensorids_columns_w_sensor_readings_60_mins__mean = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )
df_sensorids_columns_w_sensor_readings_60_mins__median = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )

In [18]:
# test that the index works 
## df_sensorids_columns_w_sensor_readings_60_mins__median[ df_sensorids_columns_w_sensor_readings_60_mins__median.index > '2018-12-01 00:00:00' ]

In [19]:
# just double-check the type … so we don't have a Series, by accident
## type( df_sensorids_columns_w_sensor_readings_2pt5_mins__mean )

In [20]:
# df_sensorids_columns_w_sensor_readings_2pt5_mins__mean['test'] = 0
## df_sensorids_columns_w_sensor_readings_2pt5_mins__mean

In [21]:
## df_sensorids_columns_w_sensor_readings_2pt5_mins__mean[ df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.index > '2018-12-01' ]

#### now do the uptime tables 

In [22]:
#### setup data frames for sensor data and sensor uptime data - 
##### - basically same length as the data period mentioned above 

In [23]:
df_sensorids_columns_w_sensor_uptime_info_2pt5_mins = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__2pt5_mins ) )

# as above but in hour frequency 
df_sensorids_columns_w_sensor_uptime_info_60_mins = pd.DataFrame( index=pd.DatetimeIndex( time_date_period_as_date_range__60_mins ) )

In [24]:
# test 
## df_sensorids_columns_w_sensor_uptime_info_2pt5_mins

In [25]:
# test 
## df_sensorids_columns_w_sensor_uptime_info_60_mins

#### load data 

In [33]:
# load ld data 

# %time df_sensorData = pd.read_csv( stockholm_2019_url )

%time df_sensorData = pd.read_csv( curr_in_data_url )
curr_in_data_url

CPU times: user 13.1 s, sys: 1.48 s, total: 14.6 s
Wall time: 11.4 s


'/mnt/virtio-bbc6cf3a-042b-4410-9/luftdaten/luftdaten_daten/uncleaned_ld_daten/historic_sds011/exported_data/ld_stkhlm_data_20200510_exp.csv'

In [34]:
# check columns 
df_sensorData.columns

Index(['sensor_id', 'lat', 'lon', 'timestamp', 'p1', 'p2', 'fixed_p1_value',
       'fixed_p2_value', 'fixed_latlon_values', 'humidity__local_dht22',
       'humidity__local_bme280', 'humidity__smhi', 'humidity__met_no'],
      dtype='object')

In [35]:
# reduce columns for efficiency 
%time df_sensorData = df_sensorData[ [ 'sensor_id', 'timestamp', 'p1', 'p2' ] ]

CPU times: user 220 ms, sys: 105 ms, total: 325 ms
Wall time: 324 ms


In [36]:
# check size 
df_sensorData.shape

(7629460, 4)

In [37]:
# check column types 
df_sensorData.dtypes

sensor_id      int64
timestamp     object
p1           float64
p2           float64
dtype: object

In [38]:
# make sure the timestamp column is a datetime column
%time df_sensorData.timestamp = pd.to_datetime( df_sensorData.timestamp )

CPU times: user 4.12 s, sys: 251 ms, total: 4.37 s
Wall time: 1.94 s


In [39]:
# make sure the index is time(stamp)
df_sensorData = df_sensorData.set_index( 'timestamp' )

In [40]:
# reduce in-data size to relevant time period 
df_sensorData = df_sensorData[ ( df_sensorData.index > start_timestamp ) & ( df_sensorData.index < end_timestamp ) ]
df_sensorData.shape

(5829822, 3)

In [41]:
# quick check of dataset content 

# - beginning 
df_sensorData.head()

,sensor_id,p1,p2
timestamp,,,
2019-01-01 00:00:15,14296,27.07,19.23
2019-01-01 00:00:33,17650,8.60,6.03
2019-01-01 00:00:35,19597,22.33,6.63
2019-01-01 00:00:47,16147,56.07,44.27
2019-01-01 00:00:50,12673,51.03,33.63


In [42]:
# end of dataset
df_sensorData.tail()

,sensor_id,p1,p2
timestamp,,,
2019-12-31 23:59:41,36089,7.78,2.40
2019-12-31 23:59:45,12129,3.20,1.33
2019-12-31 23:59:51,30940,8.93,2.05
2019-12-31 23:59:58,7597,7.23,5.20
2019-12-31 23:59:59,22082,2.07,1.33



#### sensor_ids - get unique sensor ids 

In [43]:
# get unique sensor_ids 
all_sensor_ids = df_sensorData.sensor_id.unique()
all_sensor_ids.shape

(50,)

In [44]:
# - select sensor_ids to use.
# -- this is where it is is possible to 
#     reduce the count of sensors used
# selectd_sensor_ids = pd.Series( [19597, 7426, 20265, 10727 ] ) 
selectd_sensor_ids = all_sensor_ids
selectd_sensor_ids

array([14296, 17650, 19597, 16147, 12673, 12137, 18820, 10723, 10823,
       12129, 14837, 14133,  7597,  9436, 12393, 12141, 14300,  9344,
       18830,  7426, 12395, 18832, 18828, 10727, 12683, 19778, 19788,
        7695, 20259, 20265, 20255, 20257, 22082, 22074, 11372, 22076,
       23858, 25657, 25862, 26131, 26423, 27221, 27734, 28028, 13485,
       30940, 31846, 31848, 35747, 36089])

In [45]:
selectd_sensor_ids.shape

(50,)

### loop : assemble df of same length sensor_id sensor values - using hourly averages

#### - the sensors might have data at the beginning and end of the selected time period, but to work with it, it's good the time length is the same. 
####.  - so we're going through the sensor values and making same length time sensor data series into a table

In [46]:
# for timing info 
total_starttime = time.time() 

# loop 
for sensorID in selectd_sensor_ids:
    print("--- working on sensor id "+str( sensorID )+" - at time "+str( time.ctime() )+" -- "+str( time.time() - total_starttime )+" s since start " )
    
    # partial timing 
    partial_starttime = time.time() 
    
    # get all the rows for this sensorID 
    df_w_all_values_of_curr_sensor_id = df_sensorData[ df_sensorData.sensor_id == sensorID ]
    ## print("\t - got all rows for curr sensor id - shape : "+str( df_w_all_values_of_curr_sensor_id.shape ) )
    #+" - at time "+( time.time() - partial_starttime)+" s" )
    
    # make sure the timestamp is there? 
    ## df_w_all_values_of_curr_sensor_id = df_w_all_values_of_curr_sensor_id.set_index("timestamp")
    
    # reduce columns for simplicity 
    df_w_all_values_of_curr_sensor_id = df_w_all_values_of_curr_sensor_id[ ['p1', 'p2'] ]
    ##print("\t - reduced column count. things now look like this : ")
    ##print( df_w_all_values_of_curr_sensor_id )
    
    
    # add start date to data time series
    df_w_all_values_of_curr_sensor_id = in_data__START_TIME__blank_row.append( df_w_all_values_of_curr_sensor_id )
    
    # add end date to data time series
    df_w_all_values_of_curr_sensor_id = df_w_all_values_of_curr_sensor_id.append( in_data__END_TIME__blank_row )
    
    ##print("\t - added start and end dates - df looks like this now : ")
    ## print( df_w_all_values_of_curr_sensor_id )
    
    # resample to the appropriate time length 
    df_w_all_values_of_curr_sensor_id__mean = df_w_all_values_of_curr_sensor_id.resample( sensor_uptime_sampling_freq ).mean().fillna( 0 )
    df_w_all_values_of_curr_sensor_id__median = df_w_all_values_of_curr_sensor_id.resample( sensor_uptime_sampling_freq ).median().fillna( 0 )
    ## print("\t - resampled in "+str( ( time.time() - partial_starttime ) )+"s")
    
#     print( df_w_all_values_of_curr_sensor_id )

    # - add the resampled data to the out array with all the sensor data 
    df_sensorids_columns_w_sensor_readings_2pt5_mins__mean[ sensorID ] = df_w_all_values_of_curr_sensor_id__mean['p1']
    df_sensorids_columns_w_sensor_readings_2pt5_mins__median[ sensorID ] = df_w_all_values_of_curr_sensor_id__median['p1']
    
    
    
print("\n \\\\\\\\\ and that all took "+str( time.time() - total_starttime )+" s :-) ")
    
    
    
    
    

--- working on sensor id 14296 - at time Mon May 25 09:21:09 2020 -- 0.0002779960632324219 s since start 
--- working on sensor id 17650 - at time Mon May 25 09:21:09 2020 -- 0.0682973861694336 s since start 
--- working on sensor id 19597 - at time Mon May 25 09:21:09 2020 -- 0.12858128547668457 s since start 
--- working on sensor id 16147 - at time Mon May 25 09:21:09 2020 -- 0.18834781646728516 s since start 
--- working on sensor id 12673 - at time Mon May 25 09:21:09 2020 -- 0.2600226402282715 s since start 
--- working on sensor id 12137 - at time Mon May 25 09:21:09 2020 -- 0.31613636016845703 s since start 
--- working on sensor id 18820 - at time Mon May 25 09:21:09 2020 -- 0.3742222785949707 s since start 
--- working on sensor id 10723 - at time Mon May 25 09:21:09 2020 -- 0.4405844211578369 s since start 
--- working on sensor id 10823 - at time Mon May 25 09:21:09 2020 -- 0.49836111068725586 s since start 
--- working on sensor id 12129 - at time Mon May 25 09:21:09 2020 

#### check the output of the 2.5 min sensor readings - MEAN version

In [47]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,56.07,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,61.43,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,51.13,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,53.27,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,31.47,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.tail()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-12-31 23:50:00,0.0,1.73,0.0,10.07,0.0,0.0,4.63,2.67,0.0,2.20,...,0.0,0.0,0.0,0.0,3.60,11.63,0.0,0.0,1.27,6.53
2019-12-31 23:52:30,0.0,1.33,0.0,5.63,0.0,0.0,5.80,3.20,0.0,3.00,...,0.0,0.0,0.0,0.0,4.37,4.09,0.0,0.0,1.10,12.07
2019-12-31 23:55:00,0.0,1.43,0.0,4.73,0.0,0.0,2.83,1.77,0.0,3.53,...,0.0,0.0,0.0,0.0,3.63,1.35,0.0,0.0,1.33,4.95
2019-12-31 23:57:30,0.0,0.00,0.0,3.87,0.0,0.0,6.37,4.17,0.0,3.20,...,0.0,0.0,0.0,0.0,4.53,8.93,0.0,0.0,0.85,7.78
2020-01-01 00:00:00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00


#### check the output of the 2.5 min sensor readings - MEDIAN version

In [49]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__median.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,56.07,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,61.43,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,51.13,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,53.27,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,31.47,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
df_sensorids_columns_w_sensor_readings_2pt5_mins__median.tail()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-12-31 23:50:00,0.0,1.73,0.0,10.07,0.0,0.0,4.63,2.67,0.0,2.20,...,0.0,0.0,0.0,0.0,3.60,11.63,0.0,0.0,1.27,6.53
2019-12-31 23:52:30,0.0,1.33,0.0,5.63,0.0,0.0,5.80,3.20,0.0,3.00,...,0.0,0.0,0.0,0.0,4.37,4.09,0.0,0.0,1.10,12.07
2019-12-31 23:55:00,0.0,1.43,0.0,4.73,0.0,0.0,2.83,1.77,0.0,3.53,...,0.0,0.0,0.0,0.0,3.63,1.35,0.0,0.0,1.33,4.95
2019-12-31 23:57:30,0.0,0.00,0.0,3.87,0.0,0.0,6.37,4.17,0.0,3.20,...,0.0,0.0,0.0,0.0,4.53,8.93,0.0,0.0,0.85,7.78
2020-01-01 00:00:00,0.0,0.00,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.00,...,0.0,0.0,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00


### Find periods with 75% uptime in an hour, 90% uptime hours in a year 

#### setup the uptime statistics df, in the same shape as the sensor readings df
- so one can make an binary index of uptime, copying the sensor readings structure


##### how? - to find which sensors have been reporting at least 75% of each hour, 90% of the year : 
- make a equivalent dimension table where each cell with a reading gets a 1 in the cell
- convert the time period from 2.5 mins to 60 mins, and add all the cell values in each cell, such that a cell will have 24 readings per hour in ideal circumstances
- get the uptime per hour by divining the cumulative hourly cell number ( above ) by the max number of readings per hour. eg 24/24 for full uptime.
- get yearly uptime by : 
- - convert each hourly cell that has uptime of at least 75% into a 1. 
- - count the number of 1's per year compared to total number of hours in year. 
- use the annual statistics to choose which sensors (columns) should be included in the final statistics. 

##### how? - to get mean and mean sensor count readings, annually? 
- include only relevant cells by using the annual hourly index grid ( the one with a 1 for each cell that has relevant hourly uptime ) to multiply the equivalent hourly table of sensor readings. 
- - Thus each cell with relevant uptime stays the same, and non-qualifying cells get multiplied by 0 and become 0. 
- - Use pandas .median and .mean functions to summarise the values for each relevant sensor ( ie column ). 

In [51]:
# copy the sensor readings table, 
# to later make an uptime statistics table of equivalent dimensions
%time df_sensorids_columns_w_sensor_uptime_info_2pt5_mins = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.copy()

CPU times: user 125 ms, sys: 156 ms, total: 281 ms
Wall time: 123 ms


In [52]:
# convert all the cells with a sensor data reading larger than 0, into 1 
# ( then we can count the time slots with a sensor value )
%time df_sensorids_columns_w_sensor_uptime_info_2pt5_mins[ df_sensorids_columns_w_sensor_uptime_info_2pt5_mins > 0 ] = 1

CPU times: user 2.75 s, sys: 160 ms, total: 2.9 s
Wall time: 301 ms


In [53]:
# check what things look like 
df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# get statistics 
## df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.describe()

In [55]:
# rough calculation : how many 2.5m periods, out of the whole year, had measurements? 

# NOTE 90% hourly coverage ( where each hour has >75% coverage - that comes later!)
# test : get percentage of year covered by sensor - just in terms of random readings - 
( df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.sum() / df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.shape[0] ).sort_values( ascending=False )

7597     0.957301
12129    0.944473
14133    0.934242
16147    0.927112
12141    0.925657
19788    0.917371
12393    0.916092
18820    0.901432
18828    0.893570
10823    0.886216
7426     0.852645
19778    0.839960
22082    0.810860
19597    0.793851
10727    0.793052
17650    0.779815
20259    0.775952
20265    0.733753
14837    0.707835
9436     0.702575
12137    0.685080
12683    0.628617
14300    0.625392
11372    0.617900
25657    0.615351
10723    0.594955
26131    0.586969
20255    0.570964
12395    0.551986
12673    0.505839
13485    0.469994
18832    0.427143
23858    0.400745
9344     0.396193
18830    0.328076
30940    0.310253
26423    0.306106
25862    0.288835
28028    0.276759
20257    0.275950
27221    0.262190
27734    0.163712
31848    0.085178
31846    0.072084
14296    0.060050
35747    0.054504
36089    0.034499
7695     0.001969
22074    0.000200
22076    0.000067
dtype: float64

### now do the hourly calculations 
- find hourly uptime
- select only hours with sufficient uptime per hour 
- get p10 data from these

#### change sampling rate for both uptime and sensors data 


In [56]:
# test / check 
df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,27.07,8.60,22.33,56.07,51.03,31.67,40.88,21.80,33.23,9.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:02:30,29.57,19.30,16.10,61.43,51.17,29.93,38.60,20.13,27.80,5.33,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:05:00,24.57,12.70,14.47,51.13,44.57,32.00,32.47,16.80,23.83,7.43,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:07:30,25.30,8.80,15.07,53.27,43.73,23.63,33.90,18.97,23.47,7.30,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 00:10:00,25.03,6.43,15.67,31.47,42.17,22.50,29.97,16.40,27.07,6.47,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# resample sensor values into 60 min periods - sum the results - MEAN version
%time df_sensorids_columns_w_sensor_readings_60_mins__mean = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.resample( output_table_final_freq ).mean()
df_sensorids_columns_w_sensor_readings_60_mins__mean.head()


CPU times: user 98.7 ms, sys: 3.87 ms, total: 103 ms
Wall time: 102 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,18.392917,9.362917,13.369167,27.185000,29.342917,18.467917,22.897083,14.691250,16.983750,5.294167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,10.402500,7.220000,10.125208,9.344583,9.372500,9.262708,19.510417,11.597708,9.155833,4.404375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,8.292500,5.723750,9.079583,4.192083,7.779583,8.031250,10.096458,11.105833,8.049583,3.870417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,10.537083,6.585833,9.809583,10.816667,8.702083,7.798750,13.593750,4.109583,8.524583,3.947917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,10.958333,7.516667,9.865417,12.065833,8.894583,8.485208,12.706667,12.502917,10.402083,3.967500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [58]:
# TEST
# TEST - trying to calculate mean values 'manually'
# TEST

# resample sensor values into 60 min periods - sum the results - MEAN version
%time df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2 = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.resample( output_table_final_freq ).sum()
df_sensorids_columns_w_sensor_readings_60_mins__mean.head()


CPU times: user 82.8 ms, sys: 167 µs, total: 82.9 ms
Wall time: 79.9 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,18.392917,9.362917,13.369167,27.185000,29.342917,18.467917,22.897083,14.691250,16.983750,5.294167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,10.402500,7.220000,10.125208,9.344583,9.372500,9.262708,19.510417,11.597708,9.155833,4.404375,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,8.292500,5.723750,9.079583,4.192083,7.779583,8.031250,10.096458,11.105833,8.049583,3.870417,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,10.537083,6.585833,9.809583,10.816667,8.702083,7.798750,13.593750,4.109583,8.524583,3.947917,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,10.958333,7.516667,9.865417,12.065833,8.894583,8.485208,12.706667,12.502917,10.402083,3.967500,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [59]:
# resample sensor values into 60 min periods - sum the results - MEDIAN version
%time df_sensorids_columns_w_sensor_readings_60_mins__median = df_sensorids_columns_w_sensor_readings_2pt5_mins__mean.resample( output_table_final_freq ).median()
df_sensorids_columns_w_sensor_readings_60_mins__median.head()


CPU times: user 158 ms, sys: 40.1 ms, total: 199 ms
Wall time: 195 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,15.830,9.035,14.030,30.435,26.935,18.0000,20.115,14.400,14.215,5.100,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,9.835,6.615,10.265,10.520,9.450,8.7925,12.825,11.100,9.230,4.235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,7.765,5.850,8.765,2.265,7.820,8.0700,9.740,10.665,8.470,3.750,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,9.515,6.170,9.765,11.250,8.150,7.6500,11.600,0.000,8.685,3.720,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,11.130,6.485,10.270,12.835,8.500,8.5150,12.235,12.370,10.770,3.835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [60]:
# uptime 
# -- count/sum the periods with a sensor measurement, in an hour, into one number
# ( then we can make a calculation uptime per hour )
%time df_sensorids_columns_w_sensor_uptime_info_60_mins = df_sensorids_columns_w_sensor_uptime_info_2pt5_mins.resample( output_table_final_freq ).sum().fillna(0)
df_sensorids_columns_w_sensor_uptime_info_60_mins.head()

CPU times: user 162 ms, sys: 0 ns, total: 162 ms
Wall time: 133 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,24.0,24.0,24.0,19.0,24.0,24.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,24.0,23.0,24.0,20.0,24.0,24.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,24.0,24.0,24.0,12.0,24.0,24.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,24.0,24.0,24.0,21.0,24.0,24.0,24.0,9.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,23.0,22.0,24.0,23.0,24.0,23.0,24.0,24.0,24.0,24.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# get hourly uptime as a decimal 
# ( count of measurements per hour, divided by possible measurements per hour ( ca 24, for 2.5 min sampling rates ))
# -- ie the amount of readings, per hour, of a sensor
%time df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val = df_sensorids_columns_w_sensor_uptime_info_60_mins.copy() / sensor_periods_in_hour
df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val.head()

CPU times: user 122 ms, sys: 11.6 ms, total: 134 ms
Wall time: 10.4 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.000000,1.000000,1.0,0.791667,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.000000,0.958333,1.0,0.833333,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.000000,1.000000,1.0,0.500000,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.000000,1.000000,1.0,0.875000,1.0,1.000000,1.0,0.375,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,0.958333,0.916667,1.0,0.958333,1.0,0.958333,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### get uptime above a certain decimal/percent ( 75% ) per hour, as binary 

In [62]:
# make a copy first 
df_uptime_above_75pct_60min_intervals = df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val.copy()
df_uptime_above_75pct_60min_intervals.head()

,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.000000,1.000000,1.0,0.791667,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.000000,0.958333,1.0,0.833333,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.000000,1.000000,1.0,0.500000,1.0,1.000000,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.000000,1.000000,1.0,0.875000,1.0,1.000000,1.0,0.375,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,0.958333,0.916667,1.0,0.958333,1.0,0.958333,1.0,1.000,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
# then make the binary index of values above 75% coverage ( >= 0.75 )
# -- IE put a 1 where there's an hourly uptime of at least 75% 
%time df_uptime_above_75pct_60min_intervals[ df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val >= 0.75 ] = 1
%time df_uptime_above_75pct_60min_intervals[ df_sensorids_columns_w_sensor_uptime_info_60_mins__as_decimal_val < 0.75 ] = 0
df_uptime_above_75pct_60min_intervals.head()

CPU times: user 72 ms, sys: 46.2 ms, total: 118 ms
Wall time: 10.1 ms
CPU times: user 70.7 ms, sys: 49.6 ms, total: 120 ms
Wall time: 10.1 ms


,14296,17650,19597,16147,12673,12137,18820,10723,10823,12129,...,26423,27221,27734,28028,13485,30940,31846,31848,35747,36089
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 01:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 02:00:00,1.0,1.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 03:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2019-01-01 04:00:00,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [64]:
# calculate amount of hours, per year, fulfilling at least 75% uptime per hour

uptime_hours_w_75pct_or_more_sensor_coverage = ( df_uptime_above_75pct_60min_intervals.sum() / df_uptime_above_75pct_60min_intervals.shape[0] ).sort_values( ascending=False )
uptime_hours_w_75pct_or_more_sensor_coverage

7597     0.958338
12129    0.945554
14133    0.936879
12141    0.935053
16147    0.928661
19788    0.919073
12393    0.916220
17650    0.907659
18820    0.902180
18828    0.900354
10823    0.885744
7426     0.847049
19778    0.829357
22082    0.814747
10727    0.792261
19597    0.789408
20259    0.776852
20265    0.733706
14837    0.703687
12137    0.689990
9436     0.663737
11372    0.623102
25657    0.618651
14300    0.601530
26131    0.594909
10723    0.574706
20255    0.552334
12395    0.540121
12683    0.520945
12673    0.481908
13485    0.473919
18832    0.426778
9344     0.386486
23858    0.357037
30940    0.311380
18830    0.302477
26423    0.298139
25862    0.293574
28028    0.277822
20257    0.265951
27221    0.259902
27734    0.151695
31848    0.075790
31846    0.071111
14296    0.060952
35747    0.053647
36089    0.034357
7695     0.001826
22074    0.000000
22076    0.000000
dtype: float64

In [65]:
uptime_hours_w_75pct_or_more_sensor_coverage.index

Int64Index([ 7597, 12129, 14133, 12141, 16147, 19788, 12393, 17650, 18820,
            18828, 10823,  7426, 19778, 22082, 10727, 19597, 20259, 20265,
            14837, 12137,  9436, 11372, 25657, 14300, 26131, 10723, 20255,
            12395, 12683, 12673, 13485, 18832,  9344, 23858, 30940, 18830,
            26423, 25862, 28028, 20257, 27221, 27734, 31848, 31846, 14296,
            35747, 36089,  7695, 22074, 22076],
           dtype='int64')

In [66]:
# test - array access
uptime_hours_w_75pct_or_more_sensor_coverage[ 7597 ]

0.9583380892592169

In [67]:
# make a list of sensors with >= 90% coverage 
sensor_id_list_w_more_than_90pct_coverage = []

for sensor_id in uptime_hours_w_75pct_or_more_sensor_coverage.index:
    if uptime_hours_w_75pct_or_more_sensor_coverage[ sensor_id ] >= 0.9 : 
        sensor_id_list_w_more_than_90pct_coverage.append( sensor_id )
        
sensor_id_list_w_more_than_90pct_coverage

[7597, 12129, 14133, 12141, 16147, 19788, 12393, 17650, 18820, 18828]

In [68]:
# list of sensors with at least 90% sensor covarage
# ( where at least 75% of each hour has readings )
sensor_id_list_w_more_than_90pct_coverage

[7597, 12129, 14133, 12141, 16147, 19788, 12393, 17650, 18820, 18828]

### assemble relevant sensor (and uptime) data means and medians 

In [69]:
# only include columns of sensors with 75%hourly at 90% a year, coverage - uptime table
df_sensorids_columns_w_sensor_uptime_info_60_mins__w_only_relev_sensors = df_sensorids_columns_w_sensor_uptime_info_60_mins[ sensor_id_list_w_more_than_90pct_coverage ]
df_sensorids_columns_w_sensor_uptime_info_60_mins__w_only_relev_sensors.head()


,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,24.0,24.0,21.0,24.0,19.0,0.0,24.0,24.0,24.0,0.0
2019-01-01 01:00:00,24.0,24.0,21.0,24.0,20.0,0.0,24.0,23.0,24.0,0.0
2019-01-01 02:00:00,24.0,24.0,21.0,24.0,12.0,0.0,24.0,24.0,24.0,0.0
2019-01-01 03:00:00,24.0,24.0,23.0,23.0,21.0,0.0,24.0,24.0,24.0,0.0
2019-01-01 04:00:00,24.0,24.0,22.0,24.0,23.0,0.0,24.0,22.0,24.0,0.0


In [70]:
# extract only relevant sensors for binary table of hourly readings >= 75% - make binary cell index 
%time df_uptime_above_75pct_60min_intervals__w_only_relev_sensors = df_uptime_above_75pct_60min_intervals[ sensor_id_list_w_more_than_90pct_coverage ]
df_uptime_above_75pct_60min_intervals__w_only_relev_sensors.head()

CPU times: user 1.06 ms, sys: 95 µs, total: 1.16 ms
Wall time: 1.11 ms


,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 01:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 02:00:00,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 03:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
2019-01-01 04:00:00,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0


In [71]:
# only include columns with sensors with enough sensor uptime -data
# - use above table as a mask for copying only relavant data from sensors data table to new table.
%time df_sensor_data_w_only_relev_sensors__mean = %time df_sensorids_columns_w_sensor_readings_60_mins__mean[ sensor_id_list_w_more_than_90pct_coverage ]
df_sensor_data_w_only_relev_sensors__mean.head()



CPU times: user 1.26 ms, sys: 113 µs, total: 1.38 ms
Wall time: 1.33 ms
CPU times: user 2.15 ms, sys: 162 µs, total: 2.31 ms
Wall time: 2.2 ms


,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,29.777500,9.364583,27.185000,0.0,17.869167,9.362917,22.897083,0.0
2019-01-01 01:00:00,11.839167,4.404375,11.291250,7.302083,9.344583,0.0,7.527500,7.220000,19.510417,0.0
2019-01-01 02:00:00,8.600625,3.870417,8.766250,6.487083,4.192083,0.0,6.561667,5.723750,10.096458,0.0
2019-01-01 03:00:00,9.789167,3.947917,12.055833,6.877917,10.816667,0.0,7.090833,6.585833,13.593750,0.0
2019-01-01 04:00:00,11.420417,3.967500,13.006667,7.572500,12.065833,0.0,8.682083,7.516667,12.706667,0.0


### set cell with insufficient coverage ( less than 75% per hour ) to zero - mean version

## *** Annual hourly mean readings 

In [72]:
df_sensor_data_w_only_relev_sensors__with_relev_hourly_coverage__mean = df_sensor_data_w_only_relev_sensors__mean * df_uptime_above_75pct_60min_intervals__w_only_relev_sensors
df_sensor_data_w_only_relev_sensors__with_relev_hourly_coverage__mean.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,29.777500,9.364583,27.185000,0.0,17.869167,9.362917,22.897083,0.0
2019-01-01 01:00:00,11.839167,4.404375,11.291250,7.302083,9.344583,0.0,7.527500,7.220000,19.510417,0.0
2019-01-01 02:00:00,8.600625,3.870417,8.766250,6.487083,0.000000,0.0,6.561667,5.723750,10.096458,0.0
2019-01-01 03:00:00,9.789167,3.947917,12.055833,6.877917,10.816667,0.0,7.090833,6.585833,13.593750,0.0
2019-01-01 04:00:00,11.420417,3.967500,13.006667,7.572500,12.065833,0.0,8.682083,7.516667,12.706667,0.0


In [73]:
# save
%time df_sensor_data_w_only_relev_sensors__with_relev_hourly_coverage__mean.to_csv("2019_stockholm_annual_hourly_sensor_values__MEAN.csv")

CPU times: user 182 ms, sys: 24.2 ms, total: 206 ms
Wall time: 205 ms


## *** Annual hourly median readings 

In [74]:
### set all non >= 75% coverage cells to 0 / zero
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage = df_sensorids_columns_w_sensor_readings_60_mins__median * df_uptime_above_75pct_60min_intervals__w_only_relev_sensors;
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.head()

,7426,7597,7695,9344,9436,10723,10727,10823,11372,12129,...,26131,26423,27221,27734,28028,30940,31846,31848,35747,36089
2019-01-01 00:00:00,NaN,16.5200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.100,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 01:00:00,NaN,11.4500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.235,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 02:00:00,NaN,8.0075,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.750,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 03:00:00,NaN,9.0150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.720,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-01 04:00:00,NaN,11.6150,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.835,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.to_csv("2019_stockholm_annual_hourly_sensor_values__MEDIAN.csv")

## *** Per sensor annual median value

In [76]:
# calculating median for whole year, using reshape
df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.resample( "366D" ).median()

,7426,7597,7695,9344,9436,10723,10727,10823,11372,12129,...,26131,26423,27221,27734,28028,30940,31846,31848,35747,36089
2019-01-01,NaN,4.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [77]:
# doing the same as above, but in a different way
# - SPOILER : one gets the same results 
df_1_year_median_value_per_column = df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.median()
df_1_year_median_value_per_column

7426       NaN
7597     4.020
7695       NaN
9344       NaN
9436       NaN
10723      NaN
10727      NaN
10823      NaN
11372      NaN
12129    3.565
12137      NaN
12141    3.600
12393    3.635
12395      NaN
12673      NaN
12683      NaN
13485      NaN
14133    6.100
14296      NaN
14300      NaN
14837      NaN
16147    5.290
17650    2.550
18820    5.200
18828    3.700
18830      NaN
18832      NaN
19597      NaN
19778      NaN
19788    2.850
20255      NaN
20257      NaN
20259      NaN
20265      NaN
22074      NaN
22076      NaN
22082      NaN
23858      NaN
25657      NaN
25862      NaN
26131      NaN
26423      NaN
27221      NaN
27734      NaN
28028      NaN
30940      NaN
31846      NaN
31848      NaN
35747      NaN
36089      NaN
dtype: float64

## ** save : annual median value per sensor 

In [78]:
# save annual median value ( one value per sensor )
df_1_year_median_value_per_column.to_csv("2019_stockholm_annual_sensor_value__one_numer_per_sensor__MEDIAN.csv")

In [79]:
# TEST WITH REPLACE 0 with np.nan - to see if it affect median
%time df_60min_median_sensor_data__with_nan = df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.replace(0,np.nan )
df_60min_median_sensor_data__with_nan__1_YEAR_VALUE = df_sensorids_columns_w_sensor_readings_60_mins__median__w_only_cells_w_relev_coverage.resample( "366D" ).median()
df_60min_median_sensor_data__with_nan__1_YEAR_VALUE

CPU times: user 20 ms, sys: 6.04 ms, total: 26.1 ms
Wall time: 3.45 ms


,7426,7597,7695,9344,9436,10723,10727,10823,11372,12129,...,26131,26423,27221,27734,28028,30940,31846,31848,35747,36089
2019-01-01,NaN,4.02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.565,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### TEST : 'manual' mean calculation

In [80]:
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors = df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2[ sensor_id_list_w_more_than_90pct_coverage ]

In [81]:
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN = df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors / df_sensorids_columns_w_sensor_uptime_info_60_mins__w_only_relev_sensors
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,34.031429,9.364583,34.338947,NaN,17.869167,9.362917,22.897083,NaN
2019-01-01 01:00:00,11.839167,4.404375,12.904286,7.302083,11.213500,NaN,7.527500,7.533913,19.510417,NaN
2019-01-01 02:00:00,8.600625,3.870417,10.018571,6.487083,8.384167,NaN,6.561667,5.723750,10.096458,NaN
2019-01-01 03:00:00,9.789167,3.947917,12.580000,7.176957,12.361905,NaN,7.090833,6.585833,13.593750,NaN
2019-01-01 04:00:00,11.420417,3.967500,14.189091,7.572500,12.590435,NaN,8.682083,8.200000,12.706667,NaN


In [82]:
# remove cells with insufficient readings… per hour
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN =  df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN * df_uptime_above_75pct_60min_intervals__w_only_relev_sensors;

In [83]:
# the version with .mean done in the resampling from 2.5m -> 60m 
df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN.head()

,7597,12129,14133,12141,16147,19788,12393,17650,18820,18828
2019-01-01 00:00:00,19.667500,5.294167,34.031429,9.364583,34.338947,NaN,17.869167,9.362917,22.897083,NaN
2019-01-01 01:00:00,11.839167,4.404375,12.904286,7.302083,11.213500,NaN,7.527500,7.533913,19.510417,NaN
2019-01-01 02:00:00,8.600625,3.870417,10.018571,6.487083,0.000000,NaN,6.561667,5.723750,10.096458,NaN
2019-01-01 03:00:00,9.789167,3.947917,12.580000,7.176957,12.361905,NaN,7.090833,6.585833,13.593750,NaN
2019-01-01 04:00:00,11.420417,3.967500,14.189091,7.572500,12.590435,NaN,8.682083,8.200000,12.706667,NaN


### and now… find the mean/annual averages for each sensor … 
#### - remembering to divide the sum of readings by number of valid sensor readings 

In [84]:
# get the sum of sensor values, per year - MEAN
%time df_sum_of_valid_hourly_mean_sensor_values_per_year = df_sensorids_columns_w_sensor_readings_60_mins__mean__TRT2_only_relev_sensors__MEAN.sum()
df_sum_of_valid_hourly_mean_sensor_values_per_year

CPU times: user 1.39 ms, sys: 151 µs, total: 1.54 ms
Wall time: 1.35 ms


7597     67919.999327
12129    51293.818788
14133    98101.460159
12141    48196.160688
16147    83819.187854
19788    48305.342772
12393    50199.267064
17650    48952.837113
18820    81821.580742
18828    65949.806141
dtype: float64

## *** Annual mean value, per sensor

In [85]:
# get the number of readings per year 
df_uptime_count_of_valid_readings_per_year = df_uptime_above_75pct_60min_intervals__w_only_relev_sensors.sum()
df_uptime_count_of_valid_readings_per_year

7597     8396.0
12129    8284.0
14133    8208.0
12141    8192.0
16147    8136.0
19788    8052.0
12393    8027.0
17650    7952.0
18820    7904.0
18828    7888.0
dtype: float64

In [86]:
# - get the MEAN /average
mean_sensor_value_per_sensor_and_year = df_sum_of_valid_hourly_mean_sensor_values_per_year / df_uptime_count_of_valid_readings_per_year
mean_sensor_value_per_sensor_and_year

7597      8.089566
12129     6.191914
14133    11.951932
12141     5.883320
16147    10.302260
19788     5.999173
12393     6.253802
17650     6.156041
18820    10.351921
18828     8.360777
dtype: float64

## *** save : annual mean value, per sensor 

In [87]:
mean_sensor_value_per_sensor_and_year.to_csv( "2019_stockholm_annual_sensor_value__one_numer_per_sensor__MEAN.csv" )

### Fetch lat-long locations of selected sensor

In [151]:
# reload data 
%time df_sensorData = pd.read_csv( curr_in_data_url )
# reduce columns for speed
df_sensorData = df_sensorData[ ['sensor_id', 'lat', 'lon'] ]

CPU times: user 11.7 s, sys: 4.38 s, total: 16.1 s
Wall time: 12.1 s


In [155]:
# make table to export data to 
lat_lon_of_sensors_w_sufficient_uptime = pd.DataFrame( index=sensor_id_list_w_more_than_90pct_coverage )
# add lat lon columns
lat_lon_of_sensors_w_sufficient_uptime['lat'] = np.nan
lat_lon_of_sensors_w_sufficient_uptime['lon'] = np.nan
lat_lon_of_sensors_w_sufficient_uptime

,lat,lon
7597,NaN,NaN
12129,NaN,NaN
14133,NaN,NaN
12141,NaN,NaN
16147,NaN,NaN
19788,NaN,NaN
12393,NaN,NaN
17650,NaN,NaN
18820,NaN,NaN
18828,NaN,NaN


In [156]:
# loop and find the lat lons of the sensors 
for sensor_id in sensor_id_list_w_more_than_90pct_coverage:
    # fetch the first row with this sensor int the data
    first_row_of_sensor = df_sensorData[ df_sensorData.sensor_id == sensor_id ].iloc[0]
    # set the data 
    lat_lon_of_sensors_w_sufficient_uptime.loc[ sensor_id, 'lat' ] = first_row_of_sensor.lat
    lat_lon_of_sensors_w_sufficient_uptime.loc[ sensor_id, 'lon' ] = first_row_of_sensor.lon

    

## *** Sensor koordinater

In [158]:
lat_lon_of_sensors_w_sufficient_uptime

,lat,lon
7597,59.321,18.064
12129,59.341,18.040
14133,59.364,18.018
12141,59.285,18.023
16147,59.364,18.018
19788,59.332,18.243
12393,59.343,18.033
17650,59.304,18.101
18820,59.333,18.033
18828,59.302,18.099


In [ ]:
## ** sensor coordina

In [159]:
lat_lon_of_sensors_w_sufficient_uptime.to_csv("2019_stockholm__sensor_coordinates.csv")

### - WORK IN PROGRESS - get daily averages - and the 90th quantile 

In [88]:
df_sensor_values__daily_quantile_values = df_sensor_data_w_only_relev_sensors__mean.resample( "1D" ).quantile(0.9)

AttributeError: 'DatetimeIndexResampler' object has no attribute 'quantile'

In [ ]:
df_sensor_values__daily_quantile_values

In [ ]:
df_sensor_values__daily_average_values.quantile( 0.9 ) 